In [1]:
import jpype
jpype.startJVM("-Xmx4g")

import pdfplumber
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyMuPDFLoader, PDFPlumberLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, KonlpyTextSplitter
from langchain_community.retrievers import BM25Retriever, KNNRetriever
from langchain.retrievers import EnsembleRetriever
from langchain_teddynote.retrievers import KiwiBM25Retriever
from langchain.docstore.document import Document
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
import unicodedata
from konlpy.tag import Okt
import logging
import time
import re
import os
import json
from tqdm import tqdm
import torch
import os
import json
import unicodedata
from transformers import TrOCRProcessor, VisionEncoderDecoderModel, AutoTokenizer
from PIL import Image
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from pdf2image import convert_from_path
import pytesseract
# Vector stores
import fitz  # PyMuPDF



#device = 'cuda' if torch.cuda.is_available() and 'mps' in torch.cuda.get_device_capability() else 'cpu'  # GPU 사용 가능 여부 및 MPS 지원 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

AttributeError: Java package 'kr.lucypark.kkma' is not valid

In [ ]:
from langchain_unstructured import UnstructuredLoader

def get_embedding():
    device = 'cuda' if torch.cuda.is_available() and 'mps' in torch.cuda.get_device_capability() else 'cpu'  # GPU 사용 가능 여부 및 MPS 지원 여부 확인
    embeddings = HuggingFaceEmbeddings(
        model_name='jhgan/ko-sroberta-multitask',
        model_kwargs={'device': device},
        encode_kwargs={'normalize_embeddings': True})
    return embeddings
def normalize_string(s):
    try:
        normalized = unicodedata.normalize('NFC', s)
        return normalized.encode('utf-8', errors='replace').decode('utf-8')
    except Exception:
        return s
def clean_text(text):
    text = text.replace("�", "").replace("", "")  # 잘못된 인코딩 문자 제거
    text = ' '.join(text.split())  # 여러 공백을 하나로 줄임
    return text


### konlptextsplitter, pytessearct

In [ ]:
def ocr_extract_text_from_pdf(pdf_path):
    text_content = ""
    try:
        # PDF를 이미지로 변환
        images = convert_from_path(pdf_path, dpi=500)
        
        # 각 페이지에서 OCR로 텍스트 추출
        for i, image in enumerate(images):
            text = pytesseract.image_to_string(image, lang='kor+eng')
            text = clean_text(normalize_string(text))
            text_content += f"{text}\n\n"
    
    except Exception as e:
        print(f"Error extracting text from {pdf_path}: {e}")
    
    return text_content

def load_pdf_file(pdf_file):
    try:
        # OCR을 이용한 텍스트 추출
        text_content = ocr_extract_text_from_pdf(pdf_file)
        documents = [Document(page_content=text_content, metadata={"source": pdf_file})]
        return documents
    except Exception as e:
        print(f"Skipping corrupted file {pdf_file}: {e}")
        return []

def split_documents_with_konlpy(documents):
    adjusted_documents = []
    konlpy_splitter = KonlpyTextSplitter(separator='\n\n')

    for doc in documents:
        # KonlpyTextSplitter로 문장 단위 분리
        sentences = konlpy_splitter.split_text(doc.page_content)
        for sentence in sentences:
            adjusted_documents.append(Document(page_content=sentence, metadata=doc.metadata))
    
    return adjusted_documents

def make_db(df, chunk_name="train", chunk_size=800, chunk_overlap=60):
    documents = []
    chunk_file_path = f"{chunk_name}_chunks.json"

    if os.path.exists(chunk_file_path):
        print(f"Loading chunks from {chunk_file_path}...")
        documents = load_chunks(chunk_file_path)
    else:
        pdf_files = df['Source_path'].unique()
        with ThreadPoolExecutor() as executor:
            results = list(tqdm(executor.map(load_pdf_file, pdf_files), total=len(pdf_files), desc="Processing PDFs"))
        
        for result in results:
            documents.extend(result)
        
        for doc in documents:
            doc.page_content = normalize_string(doc.page_content)
        
        # KonlpyTextSplitter로 문장 단위로 먼저 분할
        documents = split_documents_with_konlpy(documents)

        # 청크 데이터를 저장
        save_chunks(documents, chunk_file_path)
    
    # make_db에서 RecursiveCharacterTextSplitter로 청크를 분할
    recursive_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "."]
    )
    documents = recursive_splitter.split_documents(documents)
    
    faiss = FAISS.from_documents(documents, embedding=get_embedding())
    bm = KiwiBM25Retriever.from_documents(documents)
    
    return faiss, bm

def save_chunks(documents, path):
    chunk_data = []
    for doc in documents:
        chunk_data.append({
            "page_content": doc.page_content,
            "metadata": doc.metadata
        })
    
    with open(path, "w", encoding="utf-8") as f:
        json.dump(chunk_data, f, ensure_ascii=False, indent=4)
    
    print(f"Chunks saved to {path}")

def load_chunks(path):
    with open(path, "r", encoding="utf-8") as f:
        chunk_data = json.load(f)
    
    documents = []
    for chunk in chunk_data:
        documents.append(Document(page_content=chunk["page_content"], metadata=chunk["metadata"]))
    
    print(f"Chunks loaded from {path}")
    return documents

In [ ]:
import gc

def clear_model_resources():
    global processor, model, tokenizer
    del processor
    del model
    del tokenizer
    gc.collect()
    torch.cuda.empty_cache()  # GPU 메모리 해제


### 언스트럭처

In [ ]:
# def normalize_string(s):
#     try:
#         normalized = unicodedata.normalize('NFC', s)
#         return normalized.encode('utf-8', errors='replace').decode('utf-8')
#     except Exception:
#         return s
# def clean_text(text):
#     text = text.replace("�", "").replace("", "")  # 잘못된 인코딩 문자 제거
#     text = ' '.join(text.split())  # 여러 공백을 하나로 줄임
#     return text

# def load_pdf_file(pdf_file):
#     try:
#         pdf_loader = UnstructuredLoader(pdf_file,
#                                         chunking_strategy = 'by_title',
#                                         max_characters = 1000,
#                                         overlap = 200)
#         return pdf_loader.load()
#     except Exception as e:
#         print(f"Skipping corrupted file {pdf_file}: {e}")
#         return []

# def make_db(df, chunk_name="train"):
#     documents = []
#     chunk_file_path = f"{chunk_name}_chunks.json"

#     if os.path.exists(chunk_file_path):
#         # 이미 저장된 청크 데이터가 있으면 불러오기
#         print(f"Loading chunks from {chunk_file_path}...")
#         documents = load_chunks(chunk_file_path)
#     else:
#         # 청크 데이터가 없으면 PDF 파일을 처리하여 새로 생성
#         pdf_files = df['Source_path'].unique()

#         # tqdm으로 파일 처리 진행 상황 표시
#         with ThreadPoolExecutor() as executor:
#             results = list(tqdm(executor.map(load_pdf_file, pdf_files), total=len(pdf_files), desc="Processing PDFs"))
        
#         for result in results:
#             documents.extend(result)
        
#         # 정규화
#         for doc in documents:
#             doc.page_content = normalize_string(doc.page_content)
        
#         # 청크 데이터를 저장
#         save_chunks(documents, chunk_file_path)
    
#     # FAISS DB 만들기 (메타데이터 포함)
#     faiss = FAISS.from_documents(documents, embedding=get_embedding())
#     bm = KiwiBM25Retriever.from_documents(documents)
    
#     return faiss, bm

# import json

# def save_chunks(documents, path):
#     chunk_data = []
#     for doc in documents:
#         chunk_data.append({
#             "page_content": doc.page_content,
#             "metadata": doc.metadata
#         })
    
#     with open(path, "w", encoding="utf-8") as f:
#         json.dump(chunk_data, f, ensure_ascii=False, indent=4)
    
#     print(f"Chunks saved to {path}")

# from langchain.docstore.document import Document

# def load_chunks(path):
#     with open(path, "r", encoding="utf-8") as f:
#         chunk_data = json.load(f)
    
#     documents = []
#     for chunk in chunk_data:
#         documents.append(Document(page_content=chunk["page_content"], metadata=chunk["metadata"]))
    
#     print(f"Chunks loaded from {path}")
#     return documents


### pdf 로더

In [ ]:
# def get_embedding():
#     device = 'cuda' if torch.cuda.is_available() and 'mps' in torch.cuda.get_device_capability() else 'cpu'  # GPU 사용 가능 여부 및 MPS 지원 여부 확인
#     embeddings = HuggingFaceEmbeddings(
#         model_name='jhgan/ko-sroberta-multitask',
#         model_kwargs={'device': device},
#         encode_kwargs={'normalize_embeddings': True})
#     return embeddings

# def normalize_string(s):
#     try:
#         normalized = unicodedata.normalize('NFC', s)
#         return normalized.encode('utf-8', errors='replace').decode('utf-8')
#     except Exception:
#         return s

# def clean_text(text):
#     text = text.replace("�", "").replace("", "")  # 잘못된 인코딩 문자 제거
#     text = ' '.join(text.split())  # 여러 공백을 하나로 줄임
#     return text

# def extract_text_and_tables(pdf_path):
#     text_content = ""
#     try:
#         # fitz를 사용한 텍스트 추출
#         doc = fitz.open(pdf_path)
#         for page in doc:
#             text = page.get_text("text")
#             if text:
#                 text = clean_text(normalize_string(text))
#                 text_content += text + "\n"

#         # pdfplumber를 사용한 표 추출
#         with pdfplumber.open(pdf_path) as pdf:
#             for page in pdf.pages:
#                 tables = page.extract_tables()
#                 if tables:
#                     text_content += "\n__table__\n"
#                     for table in tables:
#                         if table:
#                             for row in table:
#                                 processed_row = []
#                                 for cell in row:
#                                     if cell is None:
#                                         processed_row.append(" ")  # 병합된 셀 처리
#                                     else:
#                                         cell_text = normalize_string(clean_text(str(cell).strip()))
#                                         if "\n" in cell_text:
#                                             cell_text = " ".join(cell_text.split())
#                                         processed_row.append(cell_text)
#                                 row_text = " | ".join(processed_row)
#                                 text_content += row_text + "\n"
#                     text_content += "__end_table__\n"
#     except Exception as e:
#         print(f"Error extracting text and tables from {pdf_path}: {e}")
#     return text_content

# class CustomPDFLoader:
#     def __init__(self, file_path):
#         self.file_path = file_path

#     def load(self):
#         documents = []
#         try:
#             # 텍스트와 표를 모두 추출
#             text_content = extract_text_and_tables(self.file_path)

#             # 메타데이터 생성 및 Document 객체로 변환
#             metadata = {
#                 "source": self.file_path,
#             }
#             documents.append(Document(page_content=text_content, metadata=metadata))
#         except Exception as e:
#             print(f"Error loading PDF file {self.file_path}: {e}")
#         return documents

# def chunk_documents(documents):
#     adjusted_documents = []
#     for doc in documents:
#         content = doc.page_content
#         chunks = []
        
#         # 표 구분자를 기준으로 분할
#         table_sections = content.split("__table__")
        
#         for section in table_sections:
#             if "__end_table__" in section:
#                 table_content = section.split("__end_table__")[0]
#                 chunks.append(table_content)
#             else:
#                 chunk_splitter = RecursiveCharacterTextSplitter(
#                     chunk_size=1000,  # 텍스트 청크 크기를 키움
#                     chunk_overlap=200,
#                     separators=["\n\n", ".", "?", "!", "\n"]
#                 )
#                 normal_chunks = chunk_splitter.split_text(section)
#                 chunks.extend(normal_chunks)

#         for chunk in chunks:
#             adjusted_documents.append(Document(page_content=chunk, metadata=doc.metadata))
    
#     return adjusted_documents

# def format_docs(docs):
#     context = ""
#     for doc in docs:
#         context += doc.page_content
#         context += '\n\n'
#     return context

# def make_db(df):
#     documents = []
#     pdf_files = df['Source_path'].unique()

#     # tqdm으로 파일 처리 진행 상황 표시
#     with ThreadPoolExecutor() as executor:
#         results = list(tqdm(executor.map(load_pdf_file, pdf_files), total=len(pdf_files), desc="Processing PDFs"))
    
#     for result in results:
#         documents.extend(result)
    
#     # 정규화
#     for doc in documents:
#         doc.page_content = normalize_string(doc.page_content)
    
#     # 표가 망가지지 않도록 청크 분할 처리
#     chunks = chunk_documents(documents)
#     print(f"Total number of chunks: {len(chunks)}")
    
#     # FAISS DB 만들기 (메타데이터 포함)
#     faiss = FAISS.from_documents(chunks, embedding=get_embedding())
#     bm = KiwiBM25Retriever.from_documents(chunks)
    
#     return faiss, bm

# def load_pdf_file(pdf_file):
#     try:
#         pdf_loader = CustomPDFLoader(pdf_file)
#         return pdf_loader.load()
#     except Exception as e:
#         print(f"Skipping corrupted file {pdf_file}: {e}")
#         return []


### ..

In [ ]:
# 데이터 로드
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [ ]:
# train_faiss_db, train_bm_retrievier = make_db(train_df,'./train') 

In [ ]:
test_faiss_db, test_bm_retrievier = make_db(test_df,'./test')
clear_model_resources()
weight = [0.5, 0.5]
test_k = 3
test_bm_retrievier.k = test_k
#knn_retriever.k = test_k
faiss_retriever = test_faiss_db.as_retriever(search_kwargs={'k':test_k} )
test_ensemble_retriever = EnsembleRetriever(
    retrievers=[test_bm_retrievier, faiss_retriever], weights=weight , search_type="mmr",
)


Processing PDFs: 100%|██████████| 9/9 [02:23<00:00, 15.94s/it]
Created a chunk of size 6242, which is longer than the specified 4000
Created a chunk of size 6009, which is longer than the specified 4000
Created a chunk of size 18344, which is longer than the specified 4000


java.lang.OutOfMemoryError: java.lang.OutOfMemoryError: Java heap space

In [ ]:
total = []
for i in range(len(test_df)):
    query = test_df.iloc[i]['Question']
    print(f"Query: {query}")
    results = test_ensemble_retriever.invoke(query)
    chunk_sizes = [len(i.page_content) for i in results]
    total.extend(chunk_sizes)
    

Query: 2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요?
Query: 중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 무엇인가요?
Query: 중소벤처기업부의 혁신창업사업화자금(융자) 사업근거는 어떤 법률에 근거하고 있나요?
Query: 2010년에 신규 지원된 혁신창업사업화자금은 무엇인가요?
Query: 혁신창업사업화자금 중 2020년에 신규 지원된 자금은 무엇인가요?
Query: 재창업자금이 재도약지원자금으로 이관된 연도는 언제인가요?
Query: 창업기반지원과 신청 대상이 중복인 자금이 어떤 것이며, 이 자금이 폐지된 연도는 언제인가요?
Query: 혁신창업사업화자금(융자) 사업을 시행하는 주체는 누구인가요?
Query: 혁신창업사업화자금(융자) 사업 집행절차는 어떻게 되나요?
Query: 부모급여 지원 사업의 목적은 무엇인가요?
Query: 부모급여(영아수당)의 2024년 확정된 예산은 몇백만원인가요?
Query: 부모급여 지원 사업은 어떤 법령상 근거를 갖고 추진되고 있나요?
Query: 영아수당 도입에 대한 추진경위는 어떻게 되나요?
Query: 부모급여 지원사업은 언제부터 시행되었나요?
Query: 보건복지부의 부모급여(영아수당) 지원 사업시행방법은 무엇이며, 사업 수혜자는 누구인가?
Query: 노인장기요양보험 사업 운영에 대한 목적은 무엇인가요?
Query: 노인장기요양보험 운영지원에 대한 사업 내용을 설명해줘.
Query: 국고지원을 받는 기타 의료급여수급권자는 누구인가요?
Query: 장기요양보험가입자 및 피부양자의 자격취득과 관련하여 어떤 법률을 준용해야 하는가?
Query: 노인장기요양보험법이 언제 제정되고 공포되었나?
Query: 장기요양인정점수 완화가 언제 이루어졌으며, 어떤 변화가 있었나?
Query: 장기요양기관 지정갱신제의 법적 근거가 언제 마련되었는가?
Query: 22.10월에 요양보호사 1명당 시설수급자 인력배치기준이 개선된 내용은 무엇인가?
Query: 에너지 바우처 제도의 주요 내용은 무엇인가요?
Que

In [ ]:
import numpy as np
np.mean(total)

82.54901960784314

In [ ]:
import textwrap
from IPython.display import display
from IPython.display import Markdown
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

i= 3
query = test_df.iloc[i]['Question']
print(query)
print(test_df.iloc[i]['Source_path'])
results = faiss_retriever.invoke(normalize_string(query))

for i in results:
    print(i.metadata, "Size:",len(i.page_content))
    print(i.page_content)
    print('-----------------')

2010년에 신규 지원된 혁신창업사업화자금은 무엇인가요?
./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf
{'source': './test_source/보건복지부_노인장기요양보험 사업운영.pdf'} Size: 34
Page 1: Page 2: Page 3: Page 4: 변화
-----------------
{'source': './test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf'} Size: 26
Page 1: Page 2: Page 3: 협결
-----------------
{'source': './test_source/산업통상자원부_에너지바우처.pdf'} Size: 105
Page 1: Page 2: - Page 3: 협화 Page 4: " Page 5: Page 6: Page 7: Page 8: 협종화 Page 9: Page 10: 현림 화 Page 11:
-----------------


In [ ]:
import warnings

# 특정 경고 메시지를 무시하도록 설정
warnings.filterwarnings("ignore", message="No relevant docs were retrieved using the relevance score threshold")


rag_counter = {'0':0,'1':0,'2':0,'3':0,'4':0,'5':0}

for i , entry in enumerate(train_df.to_dict(orient='records')):
    # print(entry['Question'])
    # print(entry['Source_path'])
    train_retriever = train_db.as_retriever(search_type="similarity_score_threshold",
                search_kwargs={'score_threshold': 0.77,'k':2})
    rag = train_retriever.invoke(entry['Question'])

    num_rag = len(rag)
    if num_rag ==0 :
        print(i)
        print(entry['Question'])
        print(entry['Source_path'])
        rag_counter['0'] += 1
    if num_rag == 1:
        rag_counter['1'] += 1
    elif num_rag == 2:
        rag_counter['2'] += 1
    elif num_rag == 3:
        rag_counter['3'] += 1
    elif num_rag == 4:
        rag_counter['4'] += 1
    elif num_rag == 5:
        rag_counter['5'] += 1
print(rag_counter)

NameError: name 'train_db' is not defined

In [ ]:
rag_counter = {'0':0,'1':0,'2':0,'3':0,'4':0,'5':0,'6':0,'7':0,'8':0,'9':0}
test_retriver = test_db.as_retriever(search_type="similarity_score_threshold",
                search_kwargs={'score_threshold': 0.77,'k':2})
for i , entry in enumerate(test_df.to_dict(orient='records')):
    num_rag = len(test_retriver.invoke(entry['Question']))
    if num_rag ==0 :
        rag_counter['0'] += 1
    if num_rag == 1:
        rag_counter['1'] += 1
    elif num_rag == 2:
        rag_counter['2'] += 1
    elif num_rag == 3:
        rag_counter['3'] += 1
    elif num_rag == 4:
        rag_counter['4'] += 1
    elif num_rag == 5:
        rag_counter['5'] += 1
print(rag_counter)

{'0': 9, '1': 12, '2': 77, '3': 0, '4': 0, '5': 0, '6': 0, '7': 0, '8': 0, '9': 0}


In [ ]:
i = 26
querry = test_df.iloc[i]['Question']
test_retriver = test_db.as_retriever(search_kwargs={'k': 3})
print("Q: ",querry)
len(test_retriver.invoke(querry))
# print(doc_list[0].page_content)
# print(doc_list[0].metadata)

Q:  하절기바우처와 동절기바우처의 2024년 예산 규모는 각각 얼마인가요?


3

In [ ]:
test_retriver = test_db.as_retriever(search_kwargs={'search_tpye':'ngram','k': 3})
print(querry)
test_retriver.invoke(querry)

하절기바우처와 동절기바우처의 2024년 예산 규모는 각각 얼마인가요?


[Document(metadata={'source': './test_source/산업통상자원부_에너지바우처.pdf', 'page': 5}, page_content='3)’24년도 예산안 산출 근거\n① 하절기바우처 : (‘23) 36,851 백만원 → (’24) 60,950 백만원 (+24,099 백만원 )\n   *  지원대상 확대(생계·의료→생계·의료·주거·교육)와 전기요금 인상에 따른 증액\n  - (산출) 85.7만세대 x 4.3만원(‘23년) → 115.0만세대 × 5.3만원(’24년)\n② 동절기바우처 : (‘23) 129,656 백만원 → (’24) 600,521 백만원 (+470,865 백만원 )\n   *  지원대상 증가와 난방비 인상에 따른 단가 인상 등 지원규모 확대를 고려한 증액\n  - (산출) 85.3만 세대 x 15.2만원(‘23년) → ① ‘23년 동절기 지원확대 : 113.1만 세대 x 30.4만원 x 70%, \n② ‘24년 동절기 지원 : 114.6만 세대 x 31.4만원(’24년)\n③ 연탄쿠폰 : (‘23) 21,712 백만원 → (’24) 21,240 백만원 (△472백만원 )\n   * 연탄 소비가구 감소 추세에 따른 감액'),
 Document(metadata={'source': './test_source/산업통상자원부_에너지바우처.pdf', 'page': 5}, page_content='’23년 예산 ’24년 예산\n예산 산출내역 예산 산출내역\n 190,963○ 민간경상보조 (320-01 ) : 190,963 백만원\n가. 하절기바우처 (36,851 백만원 )\n ㆍ 85.7만 세대 × 4.3만원 = 36,851 백만원\n나. 동절기바우처 (129,656 백만원 )\n ㆍ 85.3만 세대 × 15.2만원 = 129,656 백만원\n다. 연탄쿠폰 (21,712 백만원 )\n ㆍ 4.6만 세대 × 47.2만원 = 21,712 백만원\n라. 등유바우처 (1,395 백만원 )\n ㆍ 0.45만 세대 × 31만원 = 1,3

In [ ]:
test_retriver = test_db.as_retriever(search_type="similarity_score_threshold",
                search_kwargs={'score_threshold': 0.8})
print(querry)
test_retriver.invoke(querry)

하절기바우처와 동절기바우처의 2024년 예산 규모는 각각 얼마인가요?


[Document(metadata={'source': './test_source/산업통상자원부_에너지바우처.pdf', 'page': 5}, page_content='3)’24년도 예산안 산출 근거\n① 하절기바우처 : (‘23) 36,851 백만원 → (’24) 60,950 백만원 (+24,099 백만원 )\n   *  지원대상 확대(생계·의료→생계·의료·주거·교육)와 전기요금 인상에 따른 증액\n  - (산출) 85.7만세대 x 4.3만원(‘23년) → 115.0만세대 × 5.3만원(’24년)\n② 동절기바우처 : (‘23) 129,656 백만원 → (’24) 600,521 백만원 (+470,865 백만원 )\n   *  지원대상 증가와 난방비 인상에 따른 단가 인상 등 지원규모 확대를 고려한 증액\n  - (산출) 85.3만 세대 x 15.2만원(‘23년) → ① ‘23년 동절기 지원확대 : 113.1만 세대 x 30.4만원 x 70%, \n② ‘24년 동절기 지원 : 114.6만 세대 x 31.4만원(’24년)\n③ 연탄쿠폰 : (‘23) 21,712 백만원 → (’24) 21,240 백만원 (△472백만원 )\n   * 연탄 소비가구 감소 추세에 따른 감액')]

In [ ]:
test_retriver = test_db.as_retriever(search_kwargs={'filter': {'source':test_df.iloc[i]['Source_path']}, 'score_threshold': 0.8, "k":3})
print(querry)
test_retriver.invoke(querry)

하절기바우처와 동절기바우처의 2024년 예산 규모는 각각 얼마인가요?


[Document(metadata={'source': './test_source/산업통상자원부_에너지바우처.pdf', 'page': 5}, page_content='3)’24년도 예산안 산출 근거\n① 하절기바우처 : (‘23) 36,851 백만원 → (’24) 60,950 백만원 (+24,099 백만원 )\n   *  지원대상 확대(생계·의료→생계·의료·주거·교육)와 전기요금 인상에 따른 증액\n  - (산출) 85.7만세대 x 4.3만원(‘23년) → 115.0만세대 × 5.3만원(’24년)\n② 동절기바우처 : (‘23) 129,656 백만원 → (’24) 600,521 백만원 (+470,865 백만원 )\n   *  지원대상 증가와 난방비 인상에 따른 단가 인상 등 지원규모 확대를 고려한 증액\n  - (산출) 85.3만 세대 x 15.2만원(‘23년) → ① ‘23년 동절기 지원확대 : 113.1만 세대 x 30.4만원 x 70%, \n② ‘24년 동절기 지원 : 114.6만 세대 x 31.4만원(’24년)\n③ 연탄쿠폰 : (‘23) 21,712 백만원 → (’24) 21,240 백만원 (△472백만원 )\n   * 연탄 소비가구 감소 추세에 따른 감액'),
 Document(metadata={'source': './test_source/산업통상자원부_에너지바우처.pdf', 'page': 5}, page_content='’23년 예산 ’24년 예산\n예산 산출내역 예산 산출내역\n 190,963○ 민간경상보조 (320-01 ) : 190,963 백만원\n가. 하절기바우처 (36,851 백만원 )\n ㆍ 85.7만 세대 × 4.3만원 = 36,851 백만원\n나. 동절기바우처 (129,656 백만원 )\n ㆍ 85.3만 세대 × 15.2만원 = 129,656 백만원\n다. 연탄쿠폰 (21,712 백만원 )\n ㆍ 4.6만 세대 × 47.2만원 = 21,712 백만원\n라. 등유바우처 (1,395 백만원 )\n ㆍ 0.45만 세대 × 31만원 = 1,3

In [ ]:
test_db.similarity_search_with_relevance_scores(querry, k=10, score_threshold=0.8,filter={'source':test_df.iloc[i]['Source_path']})


[(Document(metadata={'source': './test_source/산업통상자원부_에너지바우처.pdf', 'page': 5}, page_content='3)’24년도 예산안 산출 근거\n① 하절기바우처 : (‘23) 36,851 백만원 → (’24) 60,950 백만원 (+24,099 백만원 )\n   *  지원대상 확대(생계·의료→생계·의료·주거·교육)와 전기요금 인상에 따른 증액\n  - (산출) 85.7만세대 x 4.3만원(‘23년) → 115.0만세대 × 5.3만원(’24년)\n② 동절기바우처 : (‘23) 129,656 백만원 → (’24) 600,521 백만원 (+470,865 백만원 )\n   *  지원대상 증가와 난방비 인상에 따른 단가 인상 등 지원규모 확대를 고려한 증액\n  - (산출) 85.3만 세대 x 15.2만원(‘23년) → ① ‘23년 동절기 지원확대 : 113.1만 세대 x 30.4만원 x 70%, \n② ‘24년 동절기 지원 : 114.6만 세대 x 31.4만원(’24년)\n③ 연탄쿠폰 : (‘23) 21,712 백만원 → (’24) 21,240 백만원 (△472백만원 )\n   * 연탄 소비가구 감소 추세에 따른 감액'),
  0.8090853551046263)]

In [ ]:
test_retriver = test_db.as_retriever(search_kwargs={'filter': {'source':test_df.iloc[i]['Source_path']}, "k":3})
print(querry)
test_retriver.invoke(querry)

하절기바우처와 동절기바우처의 2024년 예산 규모는 각각 얼마인가요?


[Document(metadata={'source': './test_source/산업통상자원부_에너지바우처.pdf', 'page': 5}, page_content='3)’24년도 예산안 산출 근거\n① 하절기바우처 : (‘23) 36,851 백만원 → (’24) 60,950 백만원 (+24,099 백만원 )\n   *  지원대상 확대(생계·의료→생계·의료·주거·교육)와 전기요금 인상에 따른 증액\n  - (산출) 85.7만세대 x 4.3만원(‘23년) → 115.0만세대 × 5.3만원(’24년)\n② 동절기바우처 : (‘23) 129,656 백만원 → (’24) 600,521 백만원 (+470,865 백만원 )\n   *  지원대상 증가와 난방비 인상에 따른 단가 인상 등 지원규모 확대를 고려한 증액\n  - (산출) 85.3만 세대 x 15.2만원(‘23년) → ① ‘23년 동절기 지원확대 : 113.1만 세대 x 30.4만원 x 70%, \n② ‘24년 동절기 지원 : 114.6만 세대 x 31.4만원(’24년)\n③ 연탄쿠폰 : (‘23) 21,712 백만원 → (’24) 21,240 백만원 (△472백만원 )\n   * 연탄 소비가구 감소 추세에 따른 감액'),
 Document(metadata={'source': './test_source/산업통상자원부_에너지바우처.pdf', 'page': 5}, page_content='’23년 예산 ’24년 예산\n예산 산출내역 예산 산출내역\n 190,963○ 민간경상보조 (320-01 ) : 190,963 백만원\n가. 하절기바우처 (36,851 백만원 )\n ㆍ 85.7만 세대 × 4.3만원 = 36,851 백만원\n나. 동절기바우처 (129,656 백만원 )\n ㆍ 85.3만 세대 × 15.2만원 = 129,656 백만원\n다. 연탄쿠폰 (21,712 백만원 )\n ㆍ 4.6만 세대 × 47.2만원 = 21,712 백만원\n라. 등유바우처 (1,395 백만원 )\n ㆍ 0.45만 세대 × 31만원 = 1,3

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import EmbeddingsFilter

embeddings = get_embedding()
embeddings_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.80)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=embeddings_filter, base_retriever=test_retriver
)
compression_retriever.invoke(querry)

[_DocumentWithState(metadata={'source': './test_source/산업통상자원부_에너지바우처.pdf', 'page': 5}, page_content='3)’24년도 예산안 산출 근거\n① 하절기바우처 : (‘23) 36,851 백만원 → (’24) 60,950 백만원 (+24,099 백만원 )\n   *  지원대상 확대(생계·의료→생계·의료·주거·교육)와 전기요금 인상에 따른 증액\n  - (산출) 85.7만세대 x 4.3만원(‘23년) → 115.0만세대 × 5.3만원(’24년)\n② 동절기바우처 : (‘23) 129,656 백만원 → (’24) 600,521 백만원 (+470,865 백만원 )\n   *  지원대상 증가와 난방비 인상에 따른 단가 인상 등 지원규모 확대를 고려한 증액\n  - (산출) 85.3만 세대 x 15.2만원(‘23년) → ① ‘23년 동절기 지원확대 : 113.1만 세대 x 30.4만원 x 70%, \n② ‘24년 동절기 지원 : 114.6만 세대 x 31.4만원(’24년)\n③ 연탄쿠폰 : (‘23) 21,712 백만원 → (’24) 21,240 백만원 (△472백만원 )\n   * 연탄 소비가구 감소 추세에 따른 감액', state={'embedded_doc': [0.039121244102716446, 0.012639015913009644, -0.011476924642920494, -0.02020144648849964, 0.03877473622560501, -0.05134795606136322, -0.01866687275469303, 0.06812097877264023, 0.05506658926606178, -0.009361985139548779, 0.055208753794431686, -0.010476823896169662, -0.05019139125943184, 0.0033026474993675947, -0.05579404532909393, -0.0050392174161970615, -0.01